In [2]:
import pandas as pd

df_original = pd.read_json("http://fake-news-detector-api.herokuapp.com/links/all")

df_original[0:10]

,category_id,clickbait_title,content,count,id,title,url,verified_category_id,verified_clickbait_title
0,1,0.0,The recent mobilizations in the city of Jerada...,1,2064,The recent mobilizations in the city of Jerada...,https://www.facebook.com/thetricontinental/pho...,NaN,NaN
1,1,0.0,Célia Sanchez Manduleywas a Cuban revolutionar...,1,2063,Célia Sanchez Manduleywas a Cuban revolutionar...,https://www.facebook.com/thetricontinental/pho...,NaN,NaN
2,1,0.0,None,1,2062,$18 Trillion and Counting,https://www.usnews.com/opinion/economic-intell...,NaN,NaN
3,2,NaN,Then presidential candidate Donald Trump signs...,1,2061,Schedule of President Trump's visit to South B...,https://www.southbendtribune.com/news/politics...,NaN,NaN
4,1,1.0,"WASHINGTON (CNN) - Rachel Crooks, who alleged ...",1,2059,Trump accuser secures Democratic nomination fo...,http://www.ktvz.com/news/politics/trump-accuse...,NaN,NaN
5,1,NaN,Lolade Siyonbola’s exasperated message to the ...,1,2058,A black Yale student fell asleep in her dorm’s...,https://wapo.st/2jNZMwl?tid=ss_tw&utm_term=.a6...,NaN,NaN
6,1,NaN,"Yale graduate student Lolade Siyonbola, who wa...",1,2057,Black Yale student blasts dean for inaction af...,https://thegrio.com/2018/05/10/yale-black-grad...,NaN,NaN
7,1,0.0,The day after it was revealed Trump personal a...,1,2056,Trump Lawyer Was on Kremlin-Linked Financial F...,https://slate.com/news-and-politics/2018/05/tr...,NaN,NaN
8,1,NaN,A white woman at Yale University called police...,1,2055,Sarah Braasch: 5 Fast Facts You Need to Know,https://heavy.com/news/2018/05/sarah-braasch/,NaN,NaN
9,4,0.0,Gleisi Hoffmann dá o recado: Lula é o nosso ca...,1,2054,Gleisi Hoffmann dá o recado: Lula é o nosso ca...,https://www.facebook.com/pt.brasil/videos/1741...,NaN,NaN


In [108]:
from sklearn.model_selection import train_test_split
import numpy as np

df = df_original.copy()

df.dropna(subset=["title", "content"], inplace=True)

df = df[(df['content'].str.len() > 120) & (df['url'].str.contains('youtu.be|twitter.com|vimeo.com|facebook.com') == False)]

df["category_id"] = df['verified_category_id'].fillna(df['category_id'])
df["clickbait_title"] = df['verified_clickbait_title'].fillna(df['clickbait_title'])

df["is_clickbait"] = [ "yes" if c == 0 else "no" if c == 1 else "idk" for c in df['clickbait_title'] ]
df["positive"] = [cid == 4 for cid in df["category_id"]]

df = df[["positive", "content", "title", "count", "is_clickbait"]]
df = pd.get_dummies(df, columns=["is_clickbait"])

print("Number of biased samples", len(df[df["positive"] == True]))

df

Number of biased samples 99


,positive,content,title,count,is_clickbait_idk,is_clickbait_no,is_clickbait_yes
3,False,Then presidential candidate Donald Trump signs...,Schedule of President Trump's visit to South B...,1,1,0,0
4,False,"WASHINGTON (CNN) - Rachel Crooks, who alleged ...",Trump accuser secures Democratic nomination fo...,1,0,1,0
5,False,Lolade Siyonbola’s exasperated message to the ...,A black Yale student fell asleep in her dorm’s...,1,1,0,0
6,False,"Yale graduate student Lolade Siyonbola, who wa...",Black Yale student blasts dean for inaction af...,1,1,0,0
7,False,The day after it was revealed Trump personal a...,Trump Lawyer Was on Kremlin-Linked Financial F...,1,0,0,1
8,False,A white woman at Yale University called police...,Sarah Braasch: 5 Fast Facts You Need to Know,1,1,0,0
10,True,Romário lidera corrida eleitoral pelo governo ...,Romário lidera corrida eleitoral pelo governo ...,1,0,0,1
11,True,Pimenta: quem traça a estratégia eleitoral do ...,Pimenta: quem traça a estratégia eleitoral do ...,1,0,0,1
12,False,WASHINGTON — President Trump’s personal lawyer...,Giuliani’s Law Firm Undercuts His Statements a...,1,0,0,1
14,False,"In the waning weeks of 2016, when the intellig...",Russian Leverage Over Trump Isn’t a Theory. It...,1,1,0,0


In [113]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, make_scorer, recall_score
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

def test_classifier(clf_):
    positive_df = df[df["positive"] == True]
    negative_df = df[df["positive"] == False].apply(np.random.permutation)[0:len(positive_df)]
    balanced_df = positive_df.append(negative_df)
    
    X = balanced_df
    y = balanced_df["positive"]

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    pipeline = Pipeline([
        ('features', FeatureUnion(
            transformer_list=[
                ('title', Pipeline([
                  ('selector1', FunctionTransformer(lambda x: x['title'], validate=False)),
                  ('vect1', CountVectorizer(ngram_range=(1, 3))),
                  ('tfidf1', TfidfTransformer())
                ])),
                ('content', Pipeline([
                  ('selector2', FunctionTransformer(lambda x: x['content'], validate=False)),
                  ('vect2', CountVectorizer(ngram_range=(1, 3))),
                  ('tfidf2', TfidfTransformer())
                ])),
#                 ('is_clickbait_yes', Pipeline([
#                   ('selector', FunctionTransformer(lambda x: x[
#                       ['is_clickbait_yes', 'is_clickbait_no', 'count']
#                   ].to_dict('records'), validate=False)),
#                   ('vectorizer', DictVectorizer())
#                 ]))
            ],
            transformer_weights={
                'title': 1.0,
                'content': 0.8,
                'is_clickbait_yes': 1.0
            },
        )),
        ('clf', clf_)
    ])
    
    clf = pipeline.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    
    y_pred = clf.predict(X_test)
    avg_f1 = (f1_score(y_test, y_pred, pos_label=False) + f1_score(y_test, y_pred)) / 2
    positive_recall = recall_score(y_test, y_pred)
    
    return accuracy, avg_f1, positive_recall
    
    
def test_classifier_avg(name, clf, times=3):
    accuracies = []
    f1s = []
    positive_recalls = []
    
    for i in range(0, times):
        accuracy, f1, positive_recall = test_classifier(clf)
        accuracies.append(accuracy)
        f1s.append(f1)
        positive_recalls.append(positive_recall)
        
    
    print("accuracy", round(np.mean(accuracies), 3),
          "f1", round(np.mean(f1s), 3),
          "positive_recall", round(np.mean(positive_recalls), 3),
          "total weighted", round(np.mean(accuracies) + np.mean(f1s) + np.mean(positive_recalls) * 2, 3),
          name
         )

# np.random.seed(0)
    
test_classifier_avg("MultinomialNB", MultinomialNB())
# test_classifier_avg("BernoulliNB", BernoulliNB())
# test_classifier_avg("KNN", KNeighborsClassifier())
test_classifier_avg("SGDClassifier", SGDClassifier(max_iter=1000))
# test_classifier_avg("RandomForest", RandomForestClassifier())
# test_classifier_avg("DecisionTreeClassifier", DecisionTreeClassifier())
# test_classifier_avg("SVC", SVC(kernel='rbf', probability=True))
# test_classifier_avg("MultiLayerPerceptron", MLPClassifier(solver='adam', max_iter=100), times=1)
# test_classifier_avg("VotingClassifier", VotingClassifier(estimators=[
#             ('MultinomialNB', MultinomialNB()),
#             ("MultiLayerPerceptron", MLPClassifier(solver='adam', max_iter=1000)),
#             ("SGD", SGDClassifier(max_iter=1000))
#         ]), times=1)

accuracy 0.713 f1 0.684 positive_recall 0.863 total weighted 3.123 MultinomialNB
accuracy 0.74 f1 0.736 positive_recall 0.771 total weighted 3.017 SGDClassifier
